### **Inital setup**

In [2]:
# Install TFOD API (TF 2)

%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

!git clone https://github.com/tensorflow/models.git

%cd models/research
!pip install --upgrade pip

# Compile protos.

!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.

!cp object_detection/packages/tf1/setup.py .
!python -m pip install --use-feature=2020-resolver .

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.12.0
Cloning into 'models'...
remote: Enumerating objects: 87279, done.
remote: Counting objects: 100% (1033/1033), done.
remote: Compressing objects: 100% (489/489), done.
remote: Total 87279 (delta 606), reused 928 (delta 533), pack-reused 86246
Receiving objects: 100% (87279/87279), 599.23 MiB | 34.35 MiB/s, done.
Resolving deltas: 100% (62488/62488), done.
/content/models/research
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2

Usage:   
  /usr/bin/python3 -m pip install [options] <requirement specifier> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] -r <requirements file> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] [-e] <vcs project url> ...
  /usr/bin/python3 -m pip install [options] 

### **Gather trained model and test data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!wget -q https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz
!tar xf ShelfImages.tar.gz

In [ ]:
!ls -lh ShelfImages/test | head -10

total 101M
-rw-r--r-- 1 1001 1001 1.6M Oct 23  2019 C1_P02_N1_S5_1.JPG
-rw-r--r-- 1 1001 1001 2.3M Oct 23  2019 C1_P02_N2_S2_1.JPG
-rw-r--r-- 1 1001 1001 2.3M Oct 23  2019 C1_P02_N2_S3_1.JPG
-rw-r--r-- 1 1001 1001 1.3M Oct 23  2019 C1_P03_N1_S2_1.JPG
-rw-r--r-- 1 1001 1001 1.6M Oct 23  2019 C1_P03_N1_S3_1.JPG
-rw-r--r-- 1 1001 1001 2.4M Oct 23  2019 C1_P03_N1_S4_1.JPG
-rw-r--r-- 1 1001 1001 1.4M Oct 23  2019 C1_P03_N1_S4_2.JPG
-rw-r--r-- 1 1001 1001 1.1M Oct 23  2019 C1_P03_N2_S2_1.JPG
-rw-r--r-- 1 1001 1001 1.7M Oct 23  2019 C1_P03_N2_S3_1.JPG


### **Other imports**

In [ ]:
from imutils import paths
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import json

### **Image parsing utility**

In [ ]:
def parse_image(image_path: str) -> np.ndarray:
    """Reads an image and adds a batch dimension."""
    image = plt.imread(image_path).astype(np.uint8)
    image = np.expand_dims(image, 0)
    return image

### **Load test image paths**

In [ ]:
test_image_paths = list(paths.list_images("ShelfImages/test"))
test_image_paths[:5]

['ShelfImages/test/C2_P03_N2_S3_1.JPG',
 'ShelfImages/test/C1_P10_N1_S5_1.JPG',
 'ShelfImages/test/C3_P01_N2_S3_2.JPG',
 'ShelfImages/test/C4_P03_N1_S3_1.JPG',
 'ShelfImages/test/C2_P04_N3_S2_1.JPG']

### **Load detection graph**

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile("frozen_inference_graph.pb", "rb") as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

### **Inference utility**

In [ ]:
def run_inference_for_single_image(image, graph, min_threshold=0.6):
    """Runs detection graph on an image and parses the results."""
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                "num_detections", "detection_boxes", "detection_scores",
                "detection_classes"]:
                tensor_name = key + ":0"
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
                image_tensor = tf.get_default_graph().get_tensor_by_name("image_tensor:0")

            # Run inference
            output_dict = sess.run(tensor_dict,
                                    feed_dict={image_tensor: image})

    # Post-process the results
    output_dict["detection_scores"] = output_dict["detection_scores"][0]
    mask = output_dict["detection_scores"] > min_threshold

    return output_dict["detection_scores"][mask]

### **Run bulk inference and prepare JSON file**

In [ ]:
image_to_products = {}
for image_path in tqdm(test_image_paths):
    image_name = image_path.split("/")[-1]
    image = parse_image(image_path)
    num_products = len(run_inference_for_single_image(image, detection_graph))
    image_to_products[image_name] = num_products

json_string = json.dumps(image_to_products, indent=4)
with open("image2products.json", "w") as outfile:
    outfile.write(json_string)

In [ ]:
!head -10 image2products.json

{
    "C2_P03_N2_S3_1.JPG": 22,
    "C1_P10_N1_S5_1.JPG": 51,
    "C3_P01_N2_S3_2.JPG": 17,
    "C4_P03_N1_S3_1.JPG": 33,
    "C2_P04_N3_S2_1.JPG": 21,
    "C3_P03_N2_S4_1.JPG": 49,
    "C3_P04_N1_S5_1.JPG": 40,
    "C4_P08_N2_S2_1.JPG": 24,
    "C4_P03_N1_S4_1.JPG": 45,


This notebooks takes the model trained in `Colabs/GroceryDataset_Model_Training.ipynb` notebook and runs inference with it to determine how many products are likely to be present inside a given shelf image.

## References
* https://github.com/anirbankonar123/CorrosionDetector/blob/master/rust_localization.ipynb